In [33]:
import random
import copy
import numpy as np
import theano as T
import theano.tensor as tt
#import pymc3 as pm
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt


import rpy2
from rpy2.robjects import pandas2ri

pandas2ri.activate()
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
%%capture  --no-stderr --no-stdout --no-display
%%R

library(ggplot2)
library(reshape)
library(grid)
library(dplyr)
library(gridExtra)
library(lme4)

paper_theme <- theme_light() + theme( axis.title.x = element_text(size=18),
  axis.text.x=element_text(colour="black", 
                           size = 14), 
  axis.title.y = element_text(size = 18, vjust = 1),
  axis.text.y  = element_text(size = 14),
  strip.text=element_text(size=16),
  axis.line.x = element_line(colour = "black"), 
  axis.line.y = element_line(colour = "black"),
  legend.title=element_text(size=18),
  legend.text=element_text(size=16))  

  

paper_theme_2 <- theme_light() + theme(
                                       legend.text=element_text(size=14), 
                            legend.title=element_blank(),
                     legend.key = element_rect(color = "transparent", fill="transparent"),
                            legend.position=c(0.82,0.82),
                            strip.text=element_blank(), 

                            axis.text.x=element_text(size=16,color="black"),
                            axis.text.y=element_text(size=16,color="black"),
                            axis.title.x=element_text(face="plain", size=20),
                            axis.title.y=element_text(face="plain", size=20)) 


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following object is masked from ‘package:reshape’:

    rename


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘gridExtra’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145:

In [142]:
data = pd.read_csv("mc_data.csv")
subj_group = data.groupby(["subj","incr","n_query", "stim"])


subj_data = {}
for i, trial in subj_group:
    subj = list(trial["subj"])[0]
    if subj not in subj_data:
        subj_data[subj] = []
        
    cnt = [x.count(".") for x in list(trial["stim"])]
    
    tup = (list(trial["guess"])[0],
               list(trial["mod_pred"]), list(trial["cplx"]), list(trial["dist"]), cnt)
    subj_data[subj].append(tup)

    




In [143]:
keys = list(subj_data.keys())
subjs = []
hum_preds = []
mod_preds = []
cplxs = []
dists = []
seen = []
for key in keys:
    tups = subj_data[key]
    m1,m2,m3,m4,m5,m6 = [],[],[],[],[],[]
    if len(tups) == 160:
        for tup in tups:
            m1.append(key)
            m2.append(tup[0])
            m3.append(tup[1])
            m4.append(tup[2])
            m5.append(tup[3])
            m6.append(tup[4])
        subjs.append(m1)
        hum_preds.append(m2)
        mod_preds.append(m3)
        cplxs.append(m4)
        dists.append(m5)
        seen.append(m6)

subjs = np.array(subjs)
hum_preds = np.array(hum_preds)
mod_preds = np.array(mod_preds)
cplxs = np.array(cplxs)
dists = np.array(dists)
seen = np.array(seen)

mod_shape = cplxs.shape
hum_shape = hum_preds.shape

In [144]:
print(hum_shape)



(101, 160)


In [145]:

def compute_posterior(complexities,predictions,hum_data,dists,seen, alpha,beta,noise):
    theta = 1./beta
    probs = st.gamma.pdf(complexities,a=alpha,scale=theta)
    
    
   # print(probs[0])
    
    
   # print(dists.shape)
   # print(((noise**dists))[0])

    probs = probs * noise**dists
    probs = probs * ((1-noise)**(seen-dists))
    #for p in probs:
      #  for k in p:
          #  print(k/np.sum(k))
            
            
    norm = np.sum(probs, axis=2,keepdims=True)
    
   # print (probs/norm)
    probs = probs/norm

    probs = probs * predictions
    
    probs = np.sum(probs,axis=2)
    probs = probs + (0.5-probs)*noise
    


    posterior = st.bernoulli.logpmf(hum_data, probs)

    #print(prior)
    
    
    return(np.sum(posterior))
    
    

pr = compute_posterior(cplxs[:3], mod_preds[:3],hum_preds[:3],dists[:3],seen[:3],1,10,0.1)

print(pr)

-483.40908346024264


In [ ]:
n_steps = 10000
curr_alpha = random.random() * 10.
curr_beta = random.random() * 10.
curr_noise = random.random()*0.5
curr_posterior = compute_posterior(cplxs,mod_preds,hum_preds,dists,seen, curr_alpha,curr_beta,curr_noise)


for step in range(n_steps):
    prop_alpha = np.exp(np.log(curr_alpha)+np.random.normal(0,0.1))
    prop_beta = np.exp(np.log(curr_beta)+np.random.normal(0,0.1))
    
    prop_noise = curr_noise + (random.random()-curr_noise)*0.1
    
    prop_posterior = compute_posterior(cplxs,mod_preds,hum_preds,dists,seen, prop_alpha,prop_beta,prop_noise)
    
    if (prop_posterior > curr_posterior) or (np.exp(prop_posterior - curr_posterior) > random.random()):
        curr_posterior = prop_posterior
        curr_alpha = prop_alpha
        curr_beta = prop_beta
        curr_noise = prop_noise
        
        print(step, curr_posterior, curr_alpha,curr_beta,curr_noise)
    
    
print("DONE!")
    


0 -9649.121954904149 6.366854153976158 4.179157495712612 0.24229028407603842
1 -9629.751183953944 7.651569883968405 4.4733088367526594 0.26187762824645455
5 -9621.470386325222 7.055939637738339 3.831661095039828 0.2777785228494394
9 -9592.34823018434 6.923173358756446 3.8123772894594707 0.2554909113718712
17 -9588.139129293802 6.543222145285149 3.392732709096734 0.26389295214999753
18 -9579.225168486548 6.7160728550721815 3.421751624226188 0.24643307798840364
21 -9562.256853161547 6.302838836408896 3.569284575159507 0.2378741288898341
26 -9538.813084746285 6.145618959153052 3.3078987377412608 0.21577539370003818
31 -9519.95258487223 5.617697892643753 2.9860336699269623 0.23427764661442096
43 -9518.369985422407 5.235279673974633 2.569772593597401 0.21509045087717507
44 -9513.794927910565 5.283789487907888 2.6715642261165757 0.20651735185492243
47 -9505.938367758943 4.820397447120329 2.5703314621120845 0.19416282009567498
53 -9501.668451071082 4.929252645640072 2.5278565760757092 0.20517

2093 -9459.311271594162 1.8734913254523313 1.195036621297703 0.2247503694758988
2103 -9456.32785089147 2.0709789683670614 1.2224329176969646 0.2186193730066408
2111 -9456.723431305627 2.241035824596143 1.335720230255733 0.21621978926026778
2160 -9457.235850922618 2.1337653058020636 1.222243004599171 0.21396820236843916
2163 -9457.553367922294 2.1088208316820323 1.2084009186521951 0.2217196156599558
2187 -9457.949759362942 2.135972301529248 1.314901688234267 0.2207539298574379
2212 -9456.579982235184 1.9212048684216114 1.1524507160815916 0.21952047357342253
2214 -9456.434287116605 1.9641517385430742 1.1702436381357455 0.21569866092354367
2252 -9458.049984040568 1.9754596855201294 1.2329141945491107 0.21283087705564868
2294 -9457.594739155556 1.9133862711300078 1.1455653261024124 0.22443028140661606
2310 -9459.785353869194 1.7158539191767141 1.0786020214106025 0.22870968505822992
2317 -9461.947266219559 1.8914834599743464 1.0669857361096629 0.225862138499592
2319 -9460.368953814193 1.650

KeyboardInterrupt: 